# CRAWLING DATA

Data crawling merupakan proses dimana akan adanya  proses automatis untuk mengumpulkan dan mengindeks data dari berbagai sumber seperti situs web, database, atau dokumen. Proses ini menggunakan software atau aplikasi khusus yang disebut "crawler" untuk mengakses sumber data dan mengambil informasi yang dibutuhkan. Data yang dikumpulkan melalui crawling kemudian dapat diproses dan digunakan untuk berbagai tujuan, seperti analisis data, penelitian, atau pengembangan sistem informasi.

Namun perlu digaris bawahi untuk pengambilan data melalui suatu situs web harus tetepa menggunakan etika dan aturan didalamnya, dikarenakan hal tersebut dapat melanggar hak cipta.

## Import Library

List library yang digunakan :
1. BeautifulSoup = digunakan untuk melakukan crawling pada data berita
2. snscrape = digunakan untuk crawling data pada twitter
3. pandas
4. CSV

## Snscrape


Snscrape adalah sebuah alat yang digunakan untuk melakukan crawling data dari media sosial Twitter. Snscrape memiliki kemampuan untuk mengambil data tweet, pengguna, hashtag, dan topik yang populer pada Twitter.

In [18]:
_= !pip install snscrape

In [19]:
import pandas as pd
import snscrape.modules.twitter as sracp

In [20]:
topik = "cakimin"
tweets = []
limit = 500
for tweet in sracp.TwitterSearchScraper(topik).get_items():
  if limit==0:
    break
  else:
    tweets.append([tweet.user,tweet.content,tweet.place])
    limit-=1
df = pd.DataFrame(tweets, columns=["Akun","Tweet","Lokasi"])
df

<ipython-input-20-275649913972>:8: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.user,tweet.content,tweet.place])


,Akun,Tweet,Lokasi
0,https://twitter.com/benmikir,@es_er_em @msaid_didu @PDemokrat Mirip Trik PK...,None
1,https://twitter.com/dpwpkbjabar,Ini dia ternyata ya penjelasan dari semua mist...,None
2,https://twitter.com/rmp_officials,"Kandane kok, Cak Imin ini tokoh dengan segudan...",None
3,https://twitter.com/politikonoha,Silaturahmi Ramadhan ala Konoha\n#politik #pol...,None
4,https://twitter.com/yantanu2_08,Telunjuk #Jokowi sebagai ketetapan\n\nJokowi s...,None
...,...,...,...
495,https://twitter.com/ahmadhellishfau,#JokowiCakImin #JoinKepri2019 #NasionalisRelig...,None
496,https://twitter.com/rahcmatdelima,Saksikan pukul 18:40 di Kompas TV\nSiswa Masuk...,None
497,https://twitter.com/dpwpkbjabar,Saksikan pukul 18:40 di Kompas TV\n\n#sekolah ...,None
498,https://twitter.com/ahmadhellishfau,#JokowiCakImin #JoinKepri2019 #NasionalisRelig...,None


In [21]:
df.to_csv("twitter_cakimin.csv",index=False)

## crawling dari pta.trunojoyo.ac.id

Crawling data web dengan Beautiful Soup adalah teknik untuk mengekstrak data dari halaman web dengan menggunakan library Python yang bernama Beautiful Soup. Beautiful Soup menyediakan cara yang mudah untuk menavigasi struktur HTML dan XML dari halaman web, sehingga memungkinkan kita untuk mengambil data yang kita butuhkan

In [22]:
_= !pip install bs4

In [23]:
import requests

In [24]:
from bs4 import BeautifulSoup
import pandas as pd

In [25]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = requests.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)

In [26]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [27]:
scPta.get_berita()

In [28]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,MALIKUL HAMZAH,Perancangan sistem informasi badan kepegawaian...,Kantor badan kepegawaian kota bangkalan adalah...


In [29]:
scPta.save()

## crawling data berita

In [30]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv
import pandas as pd

hades = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [31]:
def scrape_cnbc(hal):
    global hades
    a = 1
    for page in range(1,hal):
        url_cnbc = f"https://www.cnbcindonesia.com/search?query=bangkrut&kanal=&tipe=&date="
        ge = req.get(url_cnbc,hades).text
        sop = bs(ge,'lxml')
        ul = sop.find('ul',class_='list media_rows middle thumb terbaru gtm_indeks_feed')
        li = ul.find_all('li')
        for x in li:
            link_art = x.find('article').find('a')['href']        
            ge_ = req.get(link_art,hades).text
            sop_ = bs(ge_,'lxml')
            art = sop_.find('div',class_='lm_content mt10').find('article')
            headline = art.find('h1').text
            warp = art.find_all('div',class_='detail_text')
            for x in warp:
                x = x.find_all('p')
                y  = [y.text for y in x ]
                content = ''.join(y).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                print(f'done[{a}] > {headline[0:10]}')
                a += 1
                with open('beritaCNBC.csv','a')as file:
                    wr = csv.writer(file, delimiter=',')
                    wr.writerow([headline,content])

In [32]:
scrape_cnbc(6)

done[1] > Harga Bera
done[2] > Pabrik Pak
done[3] > Parah! Pab
done[4] > THR OTW Re
done[5] > Startup RI
done[6] > Google Sus
done[7] > Sempat Boo
done[8] > Kala Jusuf
done[9] > Kalahkan O
done[10] > Bos LPS Bu
done[11] > WA Aplikas
done[12] > 7 Aplikasi
done[13] > Harga Bera
done[14] > Pabrik Pak
done[15] > Parah! Pab
done[16] > THR OTW Re
done[17] > Startup RI
done[18] > Google Sus
done[19] > Sempat Boo
done[20] > Kala Jusuf
done[21] > Kalahkan O
done[22] > Bos LPS Bu
done[23] > WA Aplikas
done[24] > 7 Aplikasi
done[25] > Harga Bera
done[26] > Pabrik Pak
done[27] > Parah! Pab
done[28] > THR OTW Re
done[29] > Startup RI
done[30] > Google Sus
done[31] > Sempat Boo
done[32] > Kala Jusuf
done[33] > Kalahkan O
done[34] > Bos LPS Bu
done[35] > WA Aplikas
done[36] > 7 Aplikasi
done[37] > Harga Bera
done[38] > Pabrik Pak
done[39] > Parah! Pab
done[40] > THR OTW Re
done[41] > Startup RI
done[42] > Google Sus
done[43] > Sempat Boo
done[44] > Kala Jusuf
done[45] > Kalahkan O
done[46] > Bos LPS 

In [33]:
df = pd.read_csv('/content/beritaCNBC.csv')
df

,Harga Beras DKK Turun Hingga Jeff Bezos Bangkrut?,"Jakarta, CNBC Indonesia - Jelang lebaran, Presiden Joko Widodo mengklaim stok pangan aman selain pasokan memadai harga beberapa kebutuhan pokok juga diklaim telah turun hingga sejak tahun lalu industri teknologi dihantam krisis yang masih berlangsung sampai sekarang hingga berujung badai PHK yang makin kencang. Jozz Bezos bangkrut ?Selengkapnya dalam program Evening Up CNBC Indonesia (Rabu, 05/04/2023) berikut ini."
0,"Pabrik Pakaian Puma Tutup & PHK Massal, Ini Re...","Jakarta, CNBC Indonesia - Pemutusan Hubungan K..."
1,Parah! Pabrik Tekstil Ternyata Diam-diam Masih...,"Jakarta, CNBC Indonesia - Gelombang Pemutusan ..."
2,"THR OTW Rekening, Pedagang Emas Siap-siap Pane...","Jakarta, CNBC Indonesia - Meski harga emas sed..."
3,Startup RI Dikabarkan Tutup 10 Bulan Usai Teri...,"Jakarta, CNBC Indonesia - Startup yang bergera..."
4,"Google Susah, Tunjangan 'Receh' Karyawan Juga ...","Jakarta, CNBC Indonesia - Google akan menghent..."
...,...,...
114,Kala Jusuf Hamka Selamatkan Pendiri Astra dari...,"Jakarta, CNBC Indonesia - Astra Group merupaka..."
115,"Kalahkan Obama, Elon Musk Cetak Rekor Dunia","Jakarta, CNBC Indonesia - Elon Musk mencetak r..."
116,Bos LPS Buka-bukaan Soal Situasi AS & Bank-ban...,"Jakarta, CNBC Indonesia - Ketua Dewan Komision..."
117,"WA Aplikasi Sejuta Umat Nyaris Bangkrut, Ini C...","Jakarta, CNBC Indonesia - WhatsApp tak ubahnya..."


In [34]:
df.to_excel("output.xlsx")